<a href="https://colab.research.google.com/github/dalismo/Finance_News_Sentiment/blob/agonz%2Fworking_train_set/Copy_of_ml_sentiment_analysis_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("StopWords").getOrCreate()
# spark = SparkSession.builder.appName("NaiveBayes").getOrCreate()

In [3]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover

In [6]:
# Read in data from S3 Buckets
from pyspark import SparkFiles

url = 'https://raw.githubusercontent.com/dalismo/Finance_News_Sentiment/main/Resources/train-data.csv'
# df = pd.read_csv(url,encoding='latin-1', header=None)
# url ="https://s3.amazonaws.com/dataviz-curriculum/day_2/yelp_reviews.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("train-data.csv"), sep=",", header=None)

# Show DataFrame
df.show()

+--------+--------------------+
|     _c0|                 _c1|
+--------+--------------------+
| neutral|According to Gran...|
| neutral|Technopolis plans...|
|negative|The international...|
|positive|With the new prod...|
|positive|According to the ...|
|positive|FINANCING OF ASPO...|
|positive|For the last quar...|
|positive|In the third quar...|
|positive|Operating profit ...|
|positive|Operating profit ...|
|positive|TeliaSonera TLSN ...|
|positive|STORA ENSO , NORS...|
|positive|A purchase agreem...|
|positive|Finnish Talentum ...|
|positive|Clothing retail c...|
|positive|Consolidated net ...|
|positive|Foundries divisio...|
|positive|HELSINKI ( AFX ) ...|
|positive|Incap Contract Ma...|
|positive|Its board of dire...|
+--------+--------------------+
only showing top 20 rows



In [11]:
from pyspark.sql.functions import length
# Create a length column to be used as a future feature 
data_df = df.withColumn('length', length(df['_c1']))
data_df.show()

+--------+--------------------+------+
|     _c0|                 _c1|length|
+--------+--------------------+------+
| neutral|According to Gran...|   127|
| neutral|Technopolis plans...|   190|
|negative|The international...|   228|
|positive|With the new prod...|   206|
|positive|According to the ...|   203|
|positive|FINANCING OF ASPO...|   178|
|positive|For the last quar...|   193|
|positive|In the third quar...|   125|
|positive|Operating profit ...|   122|
|positive|Operating profit ...|   103|
|positive|TeliaSonera TLSN ...|   178|
|positive|STORA ENSO , NORS...|   164|
|positive|A purchase agreem...|   207|
|positive|Finnish Talentum ...|   163|
|positive|Clothing retail c...|   143|
|positive|Consolidated net ...|   163|
|positive|Foundries divisio...|   254|
|positive|HELSINKI ( AFX ) ...|   237|
|positive|Incap Contract Ma...|   138|
|positive|Its board of dire...|   125|
+--------+--------------------+------+
only showing top 20 rows



In [12]:
df2 = data_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
df2.printSchema()
df2.show()


root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)

+--------+--------------------+------+
|   class|                text|length|
+--------+--------------------+------+
| neutral|According to Gran...|   127|
| neutral|Technopolis plans...|   190|
|negative|The international...|   228|
|positive|With the new prod...|   206|
|positive|According to the ...|   203|
|positive|FINANCING OF ASPO...|   178|
|positive|For the last quar...|   193|
|positive|In the third quar...|   125|
|positive|Operating profit ...|   122|
|positive|Operating profit ...|   103|
|positive|TeliaSonera TLSN ...|   178|
|positive|STORA ENSO , NORS...|   164|
|positive|A purchase agreem...|   207|
|positive|Finnish Talentum ...|   163|
|positive|Clothing retail c...|   143|
|positive|Consolidated net ...|   163|
|positive|Foundries divisio...|   254|
|positive|HELSINKI ( AFX ) ...|   237|
|positive|Incap Contract Ma...|   138|
|positive|Its board of d

In [20]:
# Tokenize DataFrame
tokenizer = Tokenizer(inputCol="text", outputCol="Words")

In [21]:
# Transform DataFrame
reviewed = review_data.transform(df2)
reviewed.show()

+--------+--------------------+------+--------------------+
|   class|                text|length|               Words|
+--------+--------------------+------+--------------------+
| neutral|According to Gran...|   127|[according, to, g...|
| neutral|Technopolis plans...|   190|[technopolis, pla...|
|negative|The international...|   228|[the, internation...|
|positive|With the new prod...|   206|[with, the, new, ...|
|positive|According to the ...|   203|[according, to, t...|
|positive|FINANCING OF ASPO...|   178|[financing, of, a...|
|positive|For the last quar...|   193|[for, the, last, ...|
|positive|In the third quar...|   125|[in, the, third, ...|
|positive|Operating profit ...|   122|[operating, profi...|
|positive|Operating profit ...|   103|[operating, profi...|
|positive|TeliaSonera TLSN ...|   178|[teliasonera, tls...|
|positive|STORA ENSO , NORS...|   164|[stora, enso, ,, ...|
|positive|A purchase agreem...|   207|[a, purchase, agr...|
|positive|Finnish Talentum ...|   163|[f

In [55]:
# Remove stop words
# stop_list = ["-","``"]
stopremove = StopWordsRemover(inputCol="Words", outputCol="filtered")

In [56]:
# Transform new DataFrame
newFrame = stopremove.transform(reviewed)
newFrame.show()

+--------+--------------------+------+--------------------+--------------------+
|   class|                text|length|               Words|            filtered|
+--------+--------------------+------+--------------------+--------------------+
| neutral|According to Gran...|   127|[according, to, g...|[according, gran,...|
| neutral|Technopolis plans...|   190|[technopolis, pla...|[technopolis, pla...|
|negative|The international...|   228|[the, internation...|[international, e...|
|positive|With the new prod...|   206|[with, the, new, ...|[new, production,...|
|positive|According to the ...|   203|[according, to, t...|[according, compa...|
|positive|FINANCING OF ASPO...|   178|[financing, of, a...|[financing, aspoc...|
|positive|For the last quar...|   193|[for, the, last, ...|[last, quarter, 2...|
|positive|In the third quar...|   125|[in, the, third, ...|[third, quarter, ...|
|positive|Operating profit ...|   122|[operating, profi...|[operating, profi...|
|positive|Operating profit .

In [69]:
filtered_df = newFrame.select('filtered')
filtered_df.select("filtered").write.save("filtered.json",format="json")

In [58]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer
# Create all the features to the data set
pos_neg_to_num = StringIndexer(inputCol='class',outputCol='label')
# tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
# stopremove = StopWordsRemover(inputCol='token_text',outputCol='stop_tokens')
hashingTF = HashingTF(inputCol="filtered", outputCol='hash_token')
idf = IDF(inputCol='hash_token', outputCol='idf_token')


In [59]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector

# Create feature vectors
clean_up = VectorAssembler(inputCols=['idf_token', 'length'], outputCol='features')

In [60]:
# Create a and run a data processing Pipeline
from pyspark.ml import Pipeline
data_prep_pipeline = Pipeline(stages=[pos_neg_to_num, tokenizer, stopremove, hashingTF, idf, clean_up])

In [61]:
# Fit and transform the pipeline
cleaner = data_prep_pipeline.fit(df2)
cleaned = cleaner.transform(df2)

In [62]:
# Show label and resulting features
cleaned.select(['label', 'features']).show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(262145,[32817,33...|
|  0.0|(262145,[20102,21...|
|  2.0|(262145,[4333,939...|
|  1.0|(262145,[29917,32...|
|  1.0|(262145,[3968,440...|
|  1.0|(262145,[3968,811...|
|  1.0|(262145,[5381,928...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[9972,339...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[35529,45...|
|  1.0|(262145,[20131,32...|
|  1.0|(262145,[4978,126...|
|  1.0|(262145,[33998,38...|
|  1.0|(262145,[8254,166...|
|  1.0|(262145,[11972,22...|
|  1.0|(262145,[9144,997...|
|  1.0|(262145,[2114,688...|
|  1.0|(262145,[4371,246...|
|  1.0|(262145,[12650,21...|
+-----+--------------------+
only showing top 20 rows



In [63]:
from pyspark.ml.classification import NaiveBayes
# Break data down into a training set and a testing set
training, testing = cleaned.randomSplit([0.7, 0.3])

# Create a Naive Bayes model and fit training data
nb = NaiveBayes()
predictor = nb.fit(training)

In [64]:
# Tranform the model with the testing data
test_results = predictor.transform(testing)
test_results.show()

+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|   class|                text|length|label|               Words|            filtered|          hash_token|           idf_token|            features|       rawPrediction|         probability|prediction|
+--------+--------------------+------+-----+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+
|negative|( ADP News ) - Fe...|   205|  2.0|[(, adp, news, ),...|[(, adp, news, ),...|(262144,[10391,33...|(262144,[10391,33...|(262145,[10391,33...|[-1477.3607562162...|[1.0,1.0648168896...|       0.0|
|negative|( ADP News ) - Ja...|   197|  2.0|[(, adp, news, ),...|[(, adp, news, ),...|(262144,[8254,258...|(262144,[8254,258...|(262145,[8254,258...|[-1493.4321342848...|[1.0,3.9268385796.

In [65]:
# Use the Class Evaluator for a cleaner description
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

acc_eval = MulticlassClassificationEvaluator()
acc = acc_eval.evaluate(test_results)
print("Accuracy of model at predicting reviews was: %f" % acc)

Accuracy of model at predicting reviews was: 0.566501
